In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType
from dense_correspondence.dataset.dynamic_time_contrast_dataset import DynamicTimeContrastDataset

# LOAD DATASET
dataset_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence',
                                       'dataset', 'composite',
                                       'dynamic.yaml')
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)
dataset = DynamicTimeContrastDataset(config=dataset_config)

# LOAD DESCRIPTOR NETWORK
eval_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
eval_config = utils.getDictFromYamlFilename(eval_config_filename)

utils.set_cuda_visible_devices([0])
dce = DenseCorrespondenceEvaluation(eval_config)
network_name = "sugar_closer_3"
dcn = dce.load_network_from_config(network_name)
dcn.cuda().eval()
print "loaded dcn"

# load tcn
import torch.nn as nn
import torch.nn.functional as F

class TimeEmbeddingNetwork(nn.Module):
    def __init__(self, D):
        super(TimeEmbeddingNetwork, self).__init__()
        self.conv1 = nn.Conv2d(6, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 3, 5)
        self.fc1 = nn.Linear(12768, 12768)
        self.fc2 = nn.Linear(12768, 12768)
        self.fc3 = nn.Linear(12768, D)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1,12768)
        drop = nn.Dropout(0.1)
        x = F.relu(drop(self.fc1(x)))
        x = F.relu(drop(self.fc2(x)))
        x = self.fc3(x)
        return x

D_embedding = 32
model = TimeEmbeddingNetwork(D=D_embedding).cuda()
model.load_state_dict(torch.load("/home/peteflo/code/dense_correspondence/training/tensorboard_log_dir/2019-02-23-19-41-36/norm_net.pth"))
model.eval()
print "loaded tcn"

In [ ]:
# get camera_0 rgb image for each index of a certain log
log = "2019-02-22-18-25-16"

# get nu
scene_directory = dataset.get_full_path_for_scene(log)
state_info_filename = os.path.join(scene_directory, "states.yaml")
state_info_dict = utils.getDictFromYamlFilename(state_info_filename)
image_idxs = state_info_dict.keys() # list of integers

camera_num = 0

import matplotlib.pyplot as plt

embeddings = np.zeros((len(image_idxs),D_embedding))
colors = np.zeros((len(image_idxs)))
print embeddings.shape
print colors.shape

for i in image_idxs:
    rgb = dataset.get_rgb_image(dataset.get_image_filename(log, camera_num, i, ImageType.RGB))
    rgb_tensor = dataset.rgb_image_to_tensor(rgb)
    rgb_tensor = rgb_tensor.unsqueeze(0).cuda() #N, C, H, W
    descriptor_image = dcn.forward(rgb_tensor).detach() #N, D, H, W
    stacked = torch.cat([rgb_tensor[0], descriptor_image[0]]).unsqueeze(0)
    embedding = model(stacked)[0].detach().cpu().numpy()
    embeddings[i] = embedding
    colors[i] = float(i) / float(len(image_idxs))
    
last_index = i*1

#camera_num = 1
log = "2019-02-22-18-42-22"
scene_directory = dataset.get_full_path_for_scene(log)
state_info_filename = os.path.join(scene_directory, "states.yaml")
state_info_dict = utils.getDictFromYamlFilename(state_info_filename)
image_idxs = state_info_dict.keys() # list of integers

embeddings_new = np.zeros((len(image_idxs),D_embedding))
colors_new = np.zeros((len(image_idxs)))
embeddings = np.vstack((embeddings, embeddings_new))
colors = np.hstack((colors, colors_new))

    
for i in image_idxs:
    last_index += 1
    rgb = dataset.get_rgb_image(dataset.get_image_filename(log, camera_num, i, ImageType.RGB))
    rgb_tensor = dataset.rgb_image_to_tensor(rgb)
    rgb_tensor = rgb_tensor.unsqueeze(0).cuda() #N, C, H, W
    descriptor_image = dcn.forward(rgb_tensor).detach() #N, D, H, W
    stacked = torch.cat([rgb_tensor[0], descriptor_image[0]]).unsqueeze(0)
    embedding = model(stacked)[0].detach().cpu().numpy()
    embeddings[last_index] = embedding
    colors[last_index] = float(i) / float(len(image_idxs))

In [ ]:
# Author: Jake Vanderplas -- <vanderplas@astro.washington.edu>

print(__doc__)

from time import time

import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

from sklearn import manifold, datasets

# Next line to silence pyflakes. This import is needed.
Axes3D

n_points = len(image_idxs)
X, color = embeddings, colors

In [ ]:
n_neighbors = 10
n_components = 2

fig = plt.figure(figsize=(15, 8))
plt.suptitle("Manifold Learning with %i points, %i neighbors"
             % (1000, n_neighbors), fontsize=14)

# ax = fig.add_subplot(251, projection='3d')
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
# ax.view_init(4, -72)

methods = ['standard', 'ltsa', 'hessian', 'modified']
labels = ['LLE', 'LTSA', 'Hessian LLE', 'Modified LLE']

for i, method in enumerate(methods):
    t0 = time()
    Y = manifold.LocallyLinearEmbedding(n_neighbors, n_components,
                                        eigen_solver='auto',
                                        method=method).fit_transform(X)
    t1 = time()
    print("%s: %.2g sec" % (methods[i], t1 - t0))

    ax = fig.add_subplot(252 + i)
    plt.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.Spectral)
    plt.title("%s (%.2g sec)" % (labels[i], t1 - t0))
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    plt.axis('tight')

t0 = time()
Y = manifold.Isomap(n_neighbors, n_components).fit_transform(X)
t1 = time()
print("Isomap: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(257)
plt.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.Spectral)
plt.title("Isomap (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')


t0 = time()
mds = manifold.MDS(n_components, max_iter=100, n_init=1)
Y = mds.fit_transform(X)
t1 = time()
print("MDS: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(258)
plt.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.Spectral)
plt.title("MDS (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')


t0 = time()
se = manifold.SpectralEmbedding(n_components=n_components,
                                n_neighbors=n_neighbors)
Y = se.fit_transform(X)
t1 = time()
print("SpectralEmbedding: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(259)
plt.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.Spectral)
plt.title("SpectralEmbedding (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

t0 = time()
tsne = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
Y = tsne.fit_transform(X)
t1 = time()
print("t-SNE: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(2, 5, 10)
plt.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.Spectral)
plt.title("t-SNE (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

plt.show()